# We Do Parkinson Disease prediction using LSTM Neural Network

It's time to discover my project

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  
from sklearn import metrics

import os

In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "ankle_acc_x_",
    "ankle_acc_y_",
    "ankle_acc_z_",
    "upperleg_acc_x_",
    "upperleg_acc_y_",
    "upperleg_acc_z_",
    "trunk_acc_x_",
    "trunk_acc_y_",
    "trunk_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "NON-FREEZING",
    "FREEZING"
] 

## Preparing Dataset

## New Data Loading Code

#### BATCH_SIZE here is the Batching input data

In [3]:
TRAIN = 'D:/Master\'s Project/dataset_fog_release/Strategy/strategy_1/train/'
TEST = 'D:/Master\'s Project/dataset_fog_release/Strategy/strategy_1/test/'
train1  = []
train2 = []
test1 = []
test2 = []
BATCH_SIZE = 50
# creating batches in other way
def load_New_X(X_signals_paths):
    train = []
    test = []
    batch_size = 0
    for root, dirs, filenames in os.walk(X_signals_paths):
        for fffDir in filenames:
            X_signals = []
            file = open(X_signals_paths + fffDir, 'r')
            for row in file:
                batch_signals = []
                if int(row.strip().split(' ')[1:][9]) != 0:
                    batch_signals += [row.strip().split(' ')[1:]]
                    X_signals += batch_signals
            
            file.close()
            #print(X_signals[1:5])
            train += X_signals[:2*int(len(X_signals)/3)]
            test += X_signals[2*int(len(X_signals)/3):]
            
    return train, test

train1, test1 = load_New_X(TRAIN)
train2, test2 = load_New_X(TEST)
X_train = train1 + train2
X_test = test1 + test2

## Checking out the type of data

### This batch size is used to predict ahead value of how much amount

In [4]:
print(len(train1))
print(len(X_train))
batch_size = 150

227202
312112


### Strategy 1 Splitting on featured data

In [5]:
X_train = X_train + X_test[:int(len(X_test)/2)]
X_test = X_test[int(len(X_test)/2):]

### Strategy 2 Do not Split on featured data

### Making the data to numpy array

In [6]:
X_train_batching = np.array(X_train)
X_test_batching = np.array(X_test)

### Getting only the labels from the dataset

In [7]:
y_train_new1 = []
y_test_new1 = []
y_train_new = np.array([X_train_batching[:len(X_train_batching),9]])
y_test_new = np.array([X_test_batching[:len(X_test_batching),9]])

### Reshaping the labels 

In [8]:
y_train_new = y_train_new.reshape(len(y_train_new[0]),1)
y_test_new = y_test_new.reshape(len(y_test_new[0]),1)
print(y_train_new.shape)
print(y_test_new.shape)

(390143, 1)
(78031, 1)


In [9]:
np.unique(y_train_new)

array(['1', '2'], dtype='<U6')

### Maintaing Label Consistency

In [10]:
for i in range(0,len(y_train_new)-BATCH_SIZE):
    if (i + (batch_size*2)) < len(y_train_new):
        y_train_new1 += [int(y_train_new[i + (batch_size*2)][0])-1]
    else:
        y_train_new1 += [0]
        
for i in range(0,len(y_test_new)-BATCH_SIZE):
    if (i + (batch_size*2)) < len(y_test_new):
        y_test_new1 += [int(y_test_new[i + (batch_size*2)][0])-1]
    else:
        y_test_new1 += [0]

y_train_new = y_train_new1
y_test_new = y_test_new1

### Strategy 1 modification data

In [11]:
#y_train_new = y_train_new + y_test_new[:int(len(y_test_new)/2)]

In [12]:
#y_test_new = y_test_new[int(len(y_test_new)/2):]

In [13]:
y_train_new = np.array(y_train_new)
print(y_train_new.shape)

(390093,)


In [14]:
#def reshapeLabels(y_train_new):
#    labels = []
#    for i in range(0, len(y_train_new)):
#        labels += [str(y_train_new[i][0])]
#    return labels    

#### Unused code

In [15]:
#y_train_new = reshapeLabels(y_train_new)#[int(y) for y in x[0][0] for x in y_train_new[0]]
#y_test_new = reshapeLabels(y_test_new)#y_test_new = y_test_new

### Converting labels to numpy after refactoring

In [16]:
y_train_new = np.array(y_train_new)
y_test_new = np.array(y_test_new)
print(np.unique(y_train_new))
print(np.unique(y_test_new))

[0 1]
[0 1]


In [17]:
#y_train_new = np.array([X_train_batching[:len(X_train_batching)-BATCH_SIZE,9]])
#y_test_new = np.array([X_test_batching[:len(X_test_batching)-BATCH_SIZE,9]])
y_train_new = np.array(y_train_new)
y_test_new = np.array(y_test_new)
print(y_train_new.shape)
print(y_test_new.shape)

(390093,)
(77981,)


In [18]:
y_train_new[0]

0

## Code to create Batch sized chunk of each data instance

In [19]:
# Now batching the data as with batch size 128
def createBatching(X_train,batch_size):
    transformData = []
    for i in range(0,(len(X_train)-batch_size)):
        batch_data = X_train[i:i+batch_size,0:9]
        transformData += [batch_data]
    return np.array(transformData,dtype=np.int16)
    
X_train_batching = createBatching(X_train_batching, BATCH_SIZE)

In [20]:
X_train_batching.shape

(390093, 50, 9)

### Reshaping the training batched dataset

In [21]:
X_train_batching.shape
len(X_train_batching[0][0])
X_train_batching = np.reshape(X_train_batching,(len(X_train_batching[0]),len(X_train_batching[0][0]),len(X_train_batching)))

### Reshaping the test batched dataset

In [22]:
X_train_batching.shape
X_test_batching = createBatching(X_test_batching, BATCH_SIZE)
X_test_batching = np.reshape(X_test_batching,(len(X_test_batching[0]),len(X_test_batching[0][0]),len(X_test_batching)))

In [23]:
len(y_train_new)

390093

In [24]:
### Transforming the labels now
y_train_new = np.transpose(np.array(y_train_new))
y_train_new = np.reshape(y_train_new,(len(y_train_new),1))
y_test_new = np.transpose(np.array(y_test_new))
y_test_new.shape
y_test_new = np.reshape(y_test_new,(len(y_test_new),1))

In [25]:
print(X_train_batching.shape)
print(X_test_batching.shape)
print(y_train_new.shape)
print(y_test_new.shape)

(50, 9, 390093)
(50, 9, 77981)
(390093, 1)
(77981, 1)


## Ytrain X_train training set and data

In [26]:
y_train = y_train_new
y_test = y_test_new
X_train = np.transpose(X_train_batching,(2, 0, 1))
X_test = np.transpose(X_test_batching,(2, 0, 1))
y_train.shape

(390093, 1)

In [27]:
X_test.shape

(77981, 50, 9)

In [28]:
X_train.shape

(390093, 50, 9)

## Additional Parameters:
#### Here are some core parameter definitions for the training. 
##### The whole neural network's structure could be summarised by enumerating those parameters and the fact an LSTM is used. 

In [29]:
# Input Data 

training_data_count = len(X_train)  # 5377 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep

# LSTM Neural Network's internal structure

n_hidden = BATCH_SIZE # Hidden layer num of features
n_classes = 2 # Total classes (should go up, or should go down)


# Training 
learning_rate = 0.00009      #5
lambda_loss_amount = 0.0015
training_iters = training_data_count * 1  # Loop 300 times on the dataset
batch_size = 1000
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(77981, 50, 9) (77981, 1) 264.5790086189086 645.5039841518358
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


## Utility functions for training:

In [30]:
def LSTM_RNN(_X, _weights, _biases):
    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) 
    # new shape: (n_steps*batch_size, n_input)
    
    # Linear activation
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0) 
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many to one" style classifier, 
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]
    
    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 
    #print(batch_s)
    return batch_s


def one_hot(y_):
    # Function to encode output labels from number indexes 
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    #print(len(y_))
    y_ = y_.reshape(len(y_))
    n_values = 2 #int(np.max(y_)) + 1
    y_out = np.eye(n_values)[np.array(y_, dtype=np.int32)]
    #print(y_out)
    return y_out # Returns FLOATS

In [31]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    #'hidden': tf.Variable(tf.random_normal([n_hidden, n_hidden/2])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    #'hidden': tf.Variable(tf.random_normal([n_hidden/2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
_X = tf.transpose(x, [1, 0, 2])
print(_X)
_X = tf.reshape(_X, [-1, n_input]) 
print(_X)
print(weights['hidden'])

Tensor("transpose:0", shape=(50, ?, 9), dtype=float32)
Tensor("Reshape:0", shape=(?, 9), dtype=float32)
<tf.Variable 'Variable:0' shape=(9, 50) dtype=float32_ref>


## Let's get serious and build the neural network:

In [32]:
pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [33]:
np.unique(y_train)

array([0, 1])

# Transformed new Counter

In [34]:
from collections import Counter
y_train_transformed = []
for i in range(0,len(y_train)):
    # y_counter = Counter(np.array(y_train[i]))
    y_train_transformed += [int(y_train[i])]
y_train = np.array([y_train_transformed])

y_test_transformed = []
for j in range(0,len(y_test)):
   # y_counter = Counter(np.array(y_test[i]))
    y_test_transformed += [int(y_test[j])]

y_test = np.array([y_test_transformed])

print(y_train.shape)
print(y_test.shape)

(1, 390093)
(1, 77981)


In [35]:
y_train = np.transpose(y_train)
y_train.shape

(390093, 1)

In [36]:
np.sum(y_test)

4073

In [37]:
y_test = np.transpose(y_test)

## Hooray, now train the network using General Data

In [38]:
# To keep track of training's performance
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs =         extract_batch_size(X_train, step, batch_size)
    #batch_ys = extract_batch_size(y_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))
    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs, 
            y: batch_ys
        }
    )    
    train_losses.append(loss)
    train_accuracies.append(acc)
    step += 1
    if (step*batch_size % display_iter == 0):
        print("Training iter #" + str(step*batch_size) + \
                  ":   Batch Loss = " + "{:.6f}".format(loss) + \
                  ", Accuracy = {}".format(acc))


# Save the variables to disk.
save_path = saver.save(sess, "/tmp/model.ckpt")
print("Model saved in path: %s" % save_path)

Training iter #30000:   Batch Loss = 1.511128, Accuracy = 0.7419999837875366
Training iter #60000:   Batch Loss = 1.711285, Accuracy = 0.7540000081062317
Training iter #90000:   Batch Loss = 0.797731, Accuracy = 1.0
Training iter #120000:   Batch Loss = 2.629252, Accuracy = 0.16300000250339508
Training iter #150000:   Batch Loss = 1.730312, Accuracy = 0.6010000109672546
Training iter #180000:   Batch Loss = 0.871731, Accuracy = 0.9959999918937683
Training iter #210000:   Batch Loss = 2.234921, Accuracy = 0.2199999988079071
Training iter #240000:   Batch Loss = 0.816335, Accuracy = 1.0
Training iter #270000:   Batch Loss = 1.044688, Accuracy = 0.9769999980926514
Training iter #300000:   Batch Loss = 0.811997, Accuracy = 1.0
Training iter #330000:   Batch Loss = 0.806791, Accuracy = 1.0
Training iter #360000:   Batch Loss = 0.809351, Accuracy = 1.0
Training iter #390000:   Batch Loss = 0.810693, Accuracy = 1.0
Model saved in path: /tmp/model.ckpt


In [42]:
print(tf.all_variables())

Instructions for updating:
Please use tf.global_variables instead.
[<tf.Variable 'Variable:0' shape=(9, 50) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(50, 2) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(50,) dtype=float32_ref>, <tf.Variable 'Variable_3:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(100, 200) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(100, 200) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>, <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>, <tf.Variable 'Variable/Adam:0' shape=(9, 50) dtype=float32_ref>, <tf.Variable 'Variable/Adam_1:0' shape=(9, 50) dtype=float32_ref>, <tf.Variable 'Variable_1/Adam:0' shape=(50,

### Now adding surplus layer and train the model using target data

In [41]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

#v1 = tf.get_variable("lstm_cell_1", shape = [n_input, n_hidden])
#v2 = tf.get_variable("lstm_cell_2", shape = [n_input, n_hidden])

with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt");

    # Accuracy for test data

    one_hot_predictions, accuracy, final_loss = sess.run(
        [pred, accuracy, cost],
        feed_dict={
            x: X_test,
            #y: y_test
            y: one_hot(y_test)
        }
    )

    test_losses.append(final_loss)
    test_accuracies.append(accuracy)

    print("FINAL RESULT: " + \
          "Batch Loss = {}".format(final_loss) + \
          ", Accuracy = {}".format(accuracy))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
FINAL RESULT: Batch Loss = 1.0119580030441284, Accuracy = 0.9203395843505859


In [58]:
v1 = tf.get_variable("lstm_cell_1", shape = [n_input, n_hidden],trainable=False)

ValueError: Variable lstm_cell_1 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "D:\Anaconda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\ops.py", line 1269, in __init__
    self._traceback = _extract_stack()
  File "D:\Anaconda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\ops.py", line 2506, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "D:\Anaconda\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [ ]:
# (Inline plots: )
%matplotlib inline

font = {
    'family' : 'Bitstream Vera Sans',
    'weight' : 'bold',
    'size'   : 18
}
matplotlib.rc('font', **font)

width = 12
height = 12
plt.figure(figsize=(width, height))

indep_train_axis = np.array(range(batch_size, (len(train_losses)+1)*batch_size, batch_size))
plt.plot(indep_train_axis, np.array(train_losses),     "b--", label="Train losses")
plt.plot(indep_train_axis, np.array(train_accuracies), "g--", label="Train accuracies")

indep_test_axis = np.append(
    np.array(range(batch_size, len(test_losses)*display_iter, display_iter)[:-1]),
    [training_iters]
)
plt.plot(indep_test_axis, np.array(test_losses),     "b-", label="Test losses")
plt.plot(indep_test_axis, np.array(test_accuracies), "g-", label="Test accuracies")

plt.title("Training session's progress over iterations")
plt.legend(loc='upper right', shadow=True)
plt.ylabel('Training Progress (Loss or Accuracy values)')
plt.xlabel('Training iteration')

plt.show()

## And finally, the multi-class confusion matrix and metrics!

In [ ]:
# Results

predictions = one_hot_predictions.argmax(1)

print("Testing Accuracy: {}%".format(100*accuracy))

print("")
print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

print("")
print("Confusion Matrix:")
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

print("")
print("Confusion matrix (normalised to % of total test data):")
print(normalised_confusion_matrix)
print("Note: training and testing data is not equally distributed amongst classes, ")
print("so it is normal that more than a 6th of the data is correctly classifier in the last category.")

# Plot Results: 
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix, 
    interpolation='nearest', 
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(n_classes)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
sess.close()

In [ ]:
y_test.shape